# INFO 511 Final Project
## Author: Zeb Moffat
### Research Question:
Do cancer incidence rates differ significantly between demographic groups (sex, race/ethnicity) across US states?

### Imports

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import joblib
import os

### Initial Exploration

In [13]:
data = pd.read_csv("./data/U.S._Chronic_Disease_Indicators.csv")

print(data.info()) # 309215 rows of data, 34 columns

years_start = sorted(data["YearStart"].dropna().astype(int).unique().tolist())
years_end = sorted(data["YearEnd"].dropna().astype(int).unique().tolist())

# Entry years range from 2015 to 2022
print(f"\nStarting years: {years_start}")
print(f"Ending years: {years_end}\n")

# Print topics present in the dataset
print(f"Topics in dataset: {data["Topic"].unique()}") # "Cancer" is the only topic in this data related to my research question.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309215 entries, 0 to 309214
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   YearStart                  309215 non-null  int64  
 1   YearEnd                    309215 non-null  int64  
 2   LocationAbbr               309215 non-null  object 
 3   LocationDesc               309215 non-null  object 
 4   DataSource                 309215 non-null  object 
 5   Topic                      309215 non-null  object 
 6   Question                   309215 non-null  object 
 7   Response                   0 non-null       float64
 8   DataValueUnit              309215 non-null  object 
 9   DataValueType              309215 non-null  object 
 10  DataValue                  209196 non-null  float64
 11  DataValueAlt               209196 non-null  float64
 12  DataValueFootnoteSymbol    101716 non-null  object 
 13  DataValueFootnote          10

### Data Filtering

In [14]:
# Filter to cancer only by using topic. No other topics apply to my project
# We only want to use one rate so that we can accurately compare among groups.
# Using the age-adjusted rate makes the most sense as it accounts for cancers being more prevalent
# in older people. We have 5840 rows of cancer data that is age-adjusted.
print("Cancer data by type:")
print(data[data["Topic"] == "Cancer"]["DataValueType"].value_counts())

# Take original data and filter for only rows about Cancer with age-adjusted rate.
cancerData = data[
    (data["Topic"] == "Cancer") &
    (data["DataValueType"] == "Age-adjusted Rate")
]
# Don't keep old indices, start from 0.
cancerData = cancerData.reset_index(drop=True)

# Verify there are now 5840 rows
print(f"\nRows in age-adjusted Cancer data: {len(cancerData)}")

# Remove any data with missing rates
cancerData = cancerData[cancerData["DataValue"].notna()]

# Remove any row with a footnote symbol (including ~, #, *, etc.)
# Keep only rows where DataValueFootnoteSymbol is empty
# Footnote symbol rows don't have data in them due to data supression or some other issue
cancerData = cancerData[cancerData["DataValueFootnoteSymbol"].isna()]

print(f"\nTotal rows after filtering: {len(cancerData)}")

# What types of stratification exist?
print("\nStratification Categories:")
print(cancerData["StratificationCategory1"].value_counts())

print("\nStratification Groups:")
print(cancerData["Stratification1"].value_counts())

# Locations that are present in the data
print(cancerData["LocationAbbr"].value_counts())

# Remove US (national aggregate) and PR (very little data)
# Only state and Washington D.C. data will be kept.
cancerData = cancerData[~cancerData["LocationAbbr"].isin(["US", "PR"])].copy()

# Remove "Overall" stratification. We want specific demographics only.
cancerData = cancerData[cancerData["Stratification1"] != "Overall"].copy()

# Reset index after filtering
cancerData = cancerData.reset_index(drop=True)

Cancer data by type:
DataValueType
Number                     5840
Crude Rate                 5840
Age-adjusted Rate          5840
Crude Prevalence           2419
Age-adjusted Prevalence    2419
Name: count, dtype: int64

Rows in age-adjusted Cancer data: 5840

Total rows after filtering: 4644

Stratification Categories:
StratificationCategory1
Race/Ethnicity    2772
Sex               1144
Overall            728
Name: count, dtype: int64

Stratification Groups:
Stratification1
Overall                                           728
White, non-Hispanic                               719
Female                                            624
Black, non-Hispanic                               601
Hispanic                                          568
Male                                              520
Asian or Pacific Islander, non-Hispanic           509
American Indian or Alaska Native, non-Hispanic    375
Name: count, dtype: int64
LocationAbbr
CA    106
US    106
WA    103
FL    103
OK    1

### Finished, Filtered Data

In [15]:
print(f"\n{"="*60}")
print("FINAL CLEANED DATASET")
print(f"{"="*60}")
print(f"Total rows: {len(cancerData)}")
print(f"\nDemographic groups:")
print(cancerData["Stratification1"].value_counts())
print(f"\nNumber of states: {cancerData["LocationDesc"].nunique()}")
print(f"Number of cancer types: {cancerData["Question"].nunique()}")

# Select and keep columns needed for analysis
columns_to_keep = [
    "YearStart",
    "YearEnd", 
    "LocationAbbr",
    "LocationDesc",
    "Question",                    # Cancer type
    "DataValue",                   # The cancer rate (outcome variable)
    "LowConfidenceLimit",          # Might be useful later
    "HighConfidenceLimit",         # Might be useful later
    "StratificationCategory1",     # Sex or Race/Ethnicity
    "Stratification1"              # Actual demographic group
]

cancerData = cancerData[columns_to_keep].copy()

# Since data is by sex or race/ethnicity and both are not listed together,
# the data will be split into two different datasets to train two models on.

# Model 1: Sex differences
sexData = cancerData[cancerData["StratificationCategory1"] == "Sex"].copy()
sexData = sexData.reset_index(drop=True)

# Model 2: Race/Ethnicity differences  
raceEthnicity = cancerData[cancerData["StratificationCategory1"] == "Race/Ethnicity"].copy()
raceEthnicity = raceEthnicity.reset_index(drop=True)

print(f"\n{'='*60}")
print("DATASETS FOR REGRESSION MODELS")
print(f"{'='*60}")
print(f"Sex dataset: {len(sexData)} rows")
print(f"  - Demographic groups: {sexData["Stratification1"].unique()}")
print(f"  - Locations: {sexData["LocationDesc"].nunique()}")
print(f"  - Cancer types: {sexData["Question"].nunique()}")

print(f"\nRace/Ethnicity dataset: {len(raceEthnicity)} rows")
print(f"  - Demographic groups: {raceEthnicity["Stratification1"].unique()}")
print(f"  - Locations: {raceEthnicity["LocationDesc"].nunique()}")
print(f"  - Cancer types: {raceEthnicity["Question"].nunique()}")

# save datasets
sexData.to_csv("./data/sexData.csv", index=False)
raceEthnicity.to_csv("./data/raceEthnicity.csv", index=False)


FINAL CLEANED DATASET
Total rows: 3820

Demographic groups:
Stratification1
White, non-Hispanic                               705
Female                                            610
Black, non-Hispanic                               587
Hispanic                                          554
Male                                              508
Asian or Pacific Islander, non-Hispanic           495
American Indian or Alaska Native, non-Hispanic    361
Name: count, dtype: int64

Number of states: 51
Number of cancer types: 7

DATASETS FOR REGRESSION MODELS
Sex dataset: 1118 rows
  - Demographic groups: ['Female' 'Male']
  - Locations: 51
  - Cancer types: 7

Race/Ethnicity dataset: 2702 rows
  - Demographic groups: ['Hispanic' 'White, non-Hispanic'
 'American Indian or Alaska Native, non-Hispanic'
 'Asian or Pacific Islander, non-Hispanic' 'Black, non-Hispanic']
  - Locations: 51
  - Cancer types: 7


### Model Training

In [16]:
# Rename DataValue to cancer_rate
sexData = sexData.rename(columns={'DataValue': 'cancer_rate'})
raceEthnicity = raceEthnicity.rename(columns={'DataValue': 'cancer_rate'})

# Create models directory if it doesn't exist
os.makedirs("./models", exist_ok=True)

print("="*60)
print("MODEL 1: SEX DIFFERENCES IN CANCER RATES")
print("="*60)

# ============================================================================
# MODEL 1: SEX DIFFERENCES
# ============================================================================

# Create dummy variables for categorical predictors
sex_dummies = pd.get_dummies(sexData['Stratification1'], prefix='Sex', drop_first=True)
location_dummies = pd.get_dummies(sexData['LocationDesc'], prefix='State', drop_first=True)
cancer_dummies = pd.get_dummies(sexData['Question'], prefix='Cancer', drop_first=True)

# Combine all features
X_sex = pd.concat([sex_dummies, location_dummies, cancer_dummies], axis=1)
y_sex = sexData['cancer_rate']

# Split into training (80%) and testing (20%) sets
X_sex_train, X_sex_test, y_sex_train, y_sex_test = train_test_split(
    X_sex, y_sex, test_size=0.2, random_state=1
)

print(f"\nData Split:")
print(f"  Training set: {len(X_sex_train)} observations ({len(X_sex_train)/len(X_sex)*100:.1f}%)")
print(f"  Test set: {len(X_sex_test)} observations ({len(X_sex_test)/len(X_sex)*100:.1f}%)")

print(f"\nFeature columns created: {X_sex.shape[1]}")
print(f"  - Sex categories: {sex_dummies.shape[1]}")
print(f"  - State categories: {location_dummies.shape[1]}")
print(f"  - Cancer type categories: {cancer_dummies.shape[1]}")

# Train sex model
model_sex = LinearRegression()
model_sex.fit(X_sex_train, y_sex_train)

# Make predictions on BOTH training and test sets
y_sex_train_pred = model_sex.predict(X_sex_train)
y_sex_test_pred = model_sex.predict(X_sex_test)

# Evaluate on training set
r2_sex_train = r2_score(y_sex_train, y_sex_train_pred)
rmse_sex_train = np.sqrt(mean_squared_error(y_sex_train, y_sex_train_pred))
mae_sex_train = mean_absolute_error(y_sex_train, y_sex_train_pred)

# Evaluate on test set
r2_sex_test = r2_score(y_sex_test, y_sex_test_pred)
rmse_sex_test = np.sqrt(mean_squared_error(y_sex_test, y_sex_test_pred))
mae_sex_test = mean_absolute_error(y_sex_test, y_sex_test_pred)

print(f"\nTraining Set Performance:")
print(f"  R² Score: {r2_sex_train:.4f}")
print(f"  RMSE: {rmse_sex_train:.4f} (average error in cancer rate per 100,000)")
print(f"  MAE: {mae_sex_train:.4f} (average absolute error)")

print(f"\nTest Set Performance:")
print(f"  R² Score: {r2_sex_test:.4f}")
print(f"  RMSE: {rmse_sex_test:.4f} (average error in cancer rate per 100,000)")
print(f"  MAE: {mae_sex_test:.4f} (average absolute error)")

# Get coefficients for sex variable
sex_coef_names = sex_dummies.columns
sex_coef_values = model_sex.coef_[:len(sex_coef_names)]

print(f"\nSex Coefficients:")
print(f"Reference group: Female")
for name, coef in zip(sex_coef_names, sex_coef_values):
    print(f"  {name}: {coef:+.2f} per 100,000 people")
    if coef > 0:
        print(f"    - Males have {abs(coef):.2f} higher cancer rates on average")
    else:
        print(f"    - Males have {abs(coef):.2f} lower cancer rates on average")

# Save model and metadata
joblib.dump(model_sex, './models/sex_model.pkl')
joblib.dump(X_sex.columns.tolist(), './models/sex_feature_names.pkl')
joblib.dump({
    'train_r2': r2_sex_train,
    'test_r2': r2_sex_test,
    'train_rmse': rmse_sex_train,
    'test_rmse': rmse_sex_test
}, './models/sex_model_metrics.pkl')

print(f"\nModel saved to ./models/sex_model.pkl")

print("\n" + "="*60)
print("MODEL 2: RACE/ETHNICITY DIFFERENCES IN CANCER RATES")
print("="*60)

# ============================================================================
# MODEL 2: RACE/ETHNICITY DIFFERENCES
# ============================================================================

# Create dummy variables
race_dummies = pd.get_dummies(raceEthnicity['Stratification1'], prefix='Race', drop_first=True)
location_dummies_race = pd.get_dummies(raceEthnicity['LocationDesc'], prefix='State', drop_first=True)
cancer_dummies_race = pd.get_dummies(raceEthnicity['Question'], prefix='Cancer', drop_first=True)

# Combine all features
X_race = pd.concat([race_dummies, location_dummies_race, cancer_dummies_race], axis=1)
y_race = raceEthnicity['cancer_rate']

# Split into training and testing sets
X_race_train, X_race_test, y_race_train, y_race_test = train_test_split(
    X_race, y_race, test_size=0.2, random_state=1
)

print(f"\nData Split:")
print(f"  Training set: {len(X_race_train)} observations ({len(X_race_train)/len(X_race)*100:.1f}%)")
print(f"  Test set: {len(X_race_test)} observations ({len(X_race_test)/len(X_race)*100:.1f}%)")

print(f"\nFeature columns created: {X_race.shape[1]}")
print(f"  - Race/Ethnicity categories: {race_dummies.shape[1]}")
print(f"  - State categories: {location_dummies_race.shape[1]}")
print(f"  - Cancer type categories: {cancer_dummies_race.shape[1]}")

# Train race/ethnicity model
model_race = LinearRegression()
model_race.fit(X_race_train, y_race_train)

# Make predictions
y_race_train_pred = model_race.predict(X_race_train)
y_race_test_pred = model_race.predict(X_race_test)

# Evaluate
r2_race_train = r2_score(y_race_train, y_race_train_pred)
rmse_race_train = np.sqrt(mean_squared_error(y_race_train, y_race_train_pred))
mae_race_train = mean_absolute_error(y_race_train, y_race_train_pred)

r2_race_test = r2_score(y_race_test, y_race_test_pred)
rmse_race_test = np.sqrt(mean_squared_error(y_race_test, y_race_test_pred))
mae_race_test = mean_absolute_error(y_race_test, y_race_test_pred)

print(f"\nTraining Set Performance:")
print(f"  R² Score: {r2_race_train:.4f}")
print(f"  RMSE: {rmse_race_train:.4f}")
print(f"  MAE: {mae_race_train:.4f}")

print(f"\nTest Set Performance:")
print(f"  R² Score: {r2_race_test:.4f}")
print(f"  RMSE: {rmse_race_test:.4f}")
print(f"  MAE: {mae_race_test:.4f}")

# Get coefficients for race/ethnicity variables
race_coef_names = race_dummies.columns
race_coef_values = model_race.coef_[:len(race_coef_names)]

print(f"\nRace/Ethnicity Coefficients:")
# Figure out reference group
all_race_groups = raceEthnicity['Stratification1'].unique()
included_groups = [col.replace('Race_', '') for col in race_coef_names]
reference_group = [g for g in all_race_groups if g not in included_groups][0]
print(f"Reference group: {reference_group}")

for name, coef in zip(race_coef_names, race_coef_values):
    group_name = name.replace('Race_', '')
    print(f"  {group_name}: {coef:+.2f} per 100,000 people")
    if coef > 0:
        print(f"    - {group_name} populations have {abs(coef):.2f} higher rates than {reference_group}")
    else:
        print(f"    - {group_name} populations have {abs(coef):.2f} lower rates than {reference_group}")

# Save the model
joblib.dump(model_race, './models/race_model.pkl')
joblib.dump(X_race.columns.tolist(), './models/race_feature_names.pkl')
joblib.dump({
    'train_r2': r2_race_train,
    'test_r2': r2_race_test,
    'train_rmse': rmse_race_train,
    'test_rmse': rmse_race_test
}, './models/race_model_metrics.pkl')

print(f"\nModel saved to ./models/race_model.pkl")

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)
print(f"\nSex Model (Test Set Performance):")
print(f"  - R² Score: {r2_sex_test:.4f} ({r2_sex_test*100:.2f}% variance explained)")
print(f"  - RMSE: {rmse_sex_test:.4f} per 100,000 people")
print(f"  - Interpretation: Model predictions are off by ~{rmse_sex_test:.1f} cases/deaths per 100,000 on average")

print(f"\nRace/Ethnicity Model (Test Set Performance):")
print(f"  - R² Score: {r2_race_test:.4f} ({r2_race_test*100:.2f}% variance explained)")
print(f"  - RMSE: {rmse_race_test:.4f} per 100,000 people")
print(f"  - Interpretation: Model predictions are off by ~{rmse_race_test:.1f} cases/deaths per 100,000 on average")

MODEL 1: SEX DIFFERENCES IN CANCER RATES

Data Split:
  Training set: 894 observations (80.0%)
  Test set: 224 observations (20.0%)

Feature columns created: 57
  - Sex categories: 1
  - State categories: 50
  - Cancer type categories: 6

Training Set Performance:
  R² Score: 0.9892
  RMSE: 17.2782 (average error in cancer rate per 100,000)
  MAE: 12.8875 (average absolute error)

Test Set Performance:
  R² Score: 0.9870
  RMSE: 18.4979 (average error in cancer rate per 100,000)
  MAE: 13.8208 (average absolute error)

Sex Coefficients:
Reference group: Female
  Sex_Male: +33.08 per 100,000 people
    - Males have 33.08 higher cancer rates on average

Model saved to ./models/sex_model.pkl

MODEL 2: RACE/ETHNICITY DIFFERENCES IN CANCER RATES

Data Split:
  Training set: 2161 observations (80.0%)
  Test set: 541 observations (20.0%)

Feature columns created: 60
  - Race/Ethnicity categories: 4
  - State categories: 50
  - Cancer type categories: 6

Training Set Performance:
  R² Score: 0